In [1]:
import pandas as pd
import os
import random

from teams import TEAMS_INFO, TeamKeys

In [2]:
cur_team_names = {x.name for x in TEAMS_INFO}
cur_team_names

{'ARSENAL_FC',
 'ASTON_VILLA_FC',
 'BRENTFORD_FC',
 'BRIGHTON_AND_HOVE_ALBION_FC',
 'BURNLEY_FC',
 'CHELSEA_FC',
 'CRYSTAL_PALACE_FC',
 'EVERTON_FC',
 'LEEDS_UNITED_FC',
 'LEICESTER_CITY_FC',
 'LIVERPOOL_FC',
 'MANCHESTER_CITY_FC',
 'MANCHESTER_UNITED',
 'NEWCASTLE_UNITED_FC',
 'NORWICH_FC',
 'SOUTHAMPTON_FC',
 'TOTTENHAM_HOTSPUR_FC',
 'WATFORD_FC',
 'WEST_HAM_UNITED_FC',
 'WOLVERHAMPTON_WANDERERS_FC'}

In [3]:
for filename in os.listdir('./stats/'):
    if not filename.endswith('.csv'):
        continue
        
    print(filename)
    data = pd.read_csv(f'./stats/{filename}')
    teams_set = set(data.home_team)
    teams_set = {x.upper() for x in teams_set}
    
    for team_name in teams_set:
        try:
            TeamKeys[team_name.upper()]
        except KeyError:
            print(team_name, 'not found')

17_18.csv
STOKE_CITY_FC not found
AFC_BOURNEMOUTH not found
WEST_BROMWICH_ALBION_FC not found
SWANSEA_CITY_FC not found
HUDDERSFIELD_TOWN_AFC not found
18_19.csv
CARDIFF_CITY_FC not found
AFC_BOURNEMOUTH not found
FULHAM_FC not found
HUDDERSFIELD_TOWN_AFC not found
19_20.csv
AFC_BOURNEMOUTH not found
SHEFFIELD_UNITED_FC not found
20_21.csv
FULHAM_FC not found
WEST_BROMWICH_ALBION_FC not found
SHEFFIELD_UNITED_FC not found
21_22.csv


In [4]:
for filename in os.listdir('./stats/'):
    if not filename.endswith('.csv'):
        continue
        
    print(filename)
    data = pd.read_csv(f'./stats/{filename}')
    teams_set = set(data.home_team)
    teams_set = {x.upper() for x in teams_set}
    
    for team_name in cur_team_names:
        if team_name not in teams_set:
            print(team_name, 'not found')

17_18.csv
WOLVERHAMPTON_WANDERERS_FC not found
LEEDS_UNITED_FC not found
NORWICH_FC not found
BRENTFORD_FC not found
ASTON_VILLA_FC not found
18_19.csv
LEEDS_UNITED_FC not found
NORWICH_FC not found
BRENTFORD_FC not found
ASTON_VILLA_FC not found
19_20.csv
LEEDS_UNITED_FC not found
BRENTFORD_FC not found
20_21.csv
WATFORD_FC not found
NORWICH_FC not found
BRENTFORD_FC not found
21_22.csv


In [5]:
team1 = TeamKeys.ARSENAL_FC
team2 = TeamKeys.TOTTENHAM_HOTSPUR_FC

data[((data.home_team.str.upper() == team1.name) & (data.away_team.str.upper() == team2.name)) |
     ((data.home_team.str.upper() == team2.name) & (data.away_team.str.upper() == team1.name))]




,data,time,matchday,home_team,away_team,goal_score_h_t,goal_score_a_t,at_1_half_h_t,at_1_half_a_t,corners_h_t,corners_a_t,shots_on_goal_h_t,shots_on_goal_a_t,shots_on_target_h_t,shots_on_target_a_t,yellow_cards_h_t,yellow_cards_a_t,Unnamed: 17
58,26.09.21,16:30,6,Arsenal_FC,Tottenham_Hotspur_FC,3,1,3,0,4,4,12,10,7,4,2.0,1.0,NaN


In [6]:
def extract_plot_values(data_df_list, team1, team2, prefix):
    team1_values = list()
    team2_values = list()
    
    for df in data_df_list:
        line_1_2 = df[((df.home_team.str.upper() == team1.name) &
                       (df.away_team.str.upper()  == team2.name))]
        line_2_1 = df[((df.home_team.str.upper() == team2.name) &
                       (df.away_team.str.upper()  == team1.name))]
    
        if len(line_2_1) == 0 and len(line_1_2) == 0:
            team1_values.append(None)
            team2_values.append(None)
            continue
        
        team1_value = 0
        team2_value = 0
        if len(line_1_2) != 0:
            team1_value += line_1_2[f'{prefix}_h_t'].iloc[0]
            team2_value += line_1_2[f'{prefix}_a_t'].iloc[0]
            
        print(team1_value, team2_value)
        
        
        if len(line_2_1) != 0:
            team1_value += line_2_1[f'{prefix}_a_t'].iloc[0]
            team2_value += line_2_1[f'{prefix}_h_t'].iloc[0]
            
        team1_values.append(team1_value)
        team2_values.append(team2_value)
    
    return team1_values, team2_values


In [4]:
df = pd.read_csv('stats/21_22.csv')

def aggregate_current_dataframe(df): 
    prefixes = sorted({col[:-4] for col in df.columns if col.endswith('_t')})
    teams_set = set(df.away_team) | set(df.home_team)
    teams_set = {TeamKeys[x.upper()] for x in teams_set}
    
    new_df_list = []
    
    
    for team in teams_set:
        new_df_line = dict()
        new_df_line['comands'] = TEAMS_INFO[team].russian_str
    
        for prefix in prefixes:        
            sum_value = df[df.home_team.str.upper() == team.name][f'{prefix}_h_t'].sum(0) 
            sum_value += df[df.away_team.str.upper() == team.name][f'{prefix}_a_t'].sum(0)
            count_value = ((df.home_team.str.upper() == team.name) |
                            (df.away_team.str.upper() == team.name)).sum(0)
    
            if prefix in ['yellow_cards', 'goal_score']:
                new_df_line[prefix] = round(sum_value)
            else:
                new_df_line[prefix] = round(sum_value / count_value, 1)
           
        new_df_list.append(new_df_line)
            

    return pd.DataFrame(new_df_list)

In [5]:
aggregate_current_dataframe(df)


,comands,at_1_half,corners,goal_score,shots_on_goal,shots_on_target,yellow_cards
0,Ман.Сити,1.0,8.4,20,18.3,6.0,14
1,Бёрнли,0.4,5.1,7,11.4,3.6,21
2,Астон Вилла,0.2,5.4,13,11.8,3.7,22
3,Эвертон,0.4,4.6,15,13.7,4.2,17
4,Ман.Юнайтед,0.6,6.8,16,16.7,5.8,21
5,Брентфорд,0.7,4.4,11,10.4,3.3,16
6,Тоттенхэм,0.7,5.1,9,10.4,4.4,17
7,Арсенал,0.8,5.2,10,14.6,4.6,13
8,Челси,1.2,5.8,23,14.0,5.7,9
9,Норвич,0.2,4.2,2,10.0,2.3,18
